In [4]:
import numpy as np
import theano
import theano.tensor as T
import matplotlib.pyplot as plt

In [5]:
from fer_util import getData, getBinaryData, y2indicator, error_rate, relu, init_weight_and_bias
from sklearn.utils import shuffle

In [6]:
# Creating hidden layer object so that we can create as many hidden layers as needed.

In [7]:
class HiddenLayer(object):
    # M1: input size
    # M2: output size
    # an_id: unique id to identify
    def __init__(self, M1, M2, an_id):
        self.id = an_id
        self.M1 = M1
        self.M2 = M2
        W,b = init_weight_and_bias(M1, M2)
        self.W = theano.shared(W, 'W_%s' % self.id)
        self.b = theano.shared(b, 'b_%s' % self.id)
        self.params = [self.W, self.b]
        
    def forward(self, X):
        return relu(X.dot(self.W) + self.b)

In [ ]:
class ANN(object):
    # hidden_layer_sizes: number of neurons in each hidden layer
    # eg: hidden_layer_sizes = [ #layer_1_neurons, #layer_2_neurons, #layer_3_neurons, ...]
    def __init__(self, hidden_layer_sizes):
        self.hidden_layer_sizes = hidden_layer_sizes
        
    # learning_rate, mu: momentum
    # decay for RMSProp, epochs, batch_sz: batch size, show_fig: show figure.
    def fit(self, X, Y, learning_rate=10e-7, mu=0.99, decay=0.999, epochs=400, batch_sz = 100, show_fig=False):
        # First step: Make a validation set
        X, Y = shuffle(X,Y)
        # To run the code in GPU we need it everything to be float32
        X = X.astype(np.float32)
        Y = Y.astype(np.int32)
        # Creting the testing and training set
        Xvalid, Yvalid = X[-1000:], Y[-1000:]
        X,Y = X[:-1000], Y[:-1000]
        
        # initialize the hidden layers
        N, D = X.shape # N: the number of training data, D: features of trainig data.
        K = len(set(Y)) # K: the number of distinct classes of NN.
        self.hidden_layers = []
        M1 = D
        count = 0 # unique id of the hidden layer.
        
        for M2 in self.hidden_layer_sizes:
            # Creating the hidden later
            h = HiddenLater(M1, M2, count)
            # Append it to the hidden layers of the NN
            self.hidden_layers.append(h)
            M1=M2
            count +=1
            
        # Initializing the last logistic regression layer
        W, b = init_weight_and_bias(M1, K)
        self.W = theano.shared(W, 'W_logreg')
        self.b = theano.shared(b, 'b_logreg')
        
        # Collecting the parameters of all hidden layers
        self.params = [self.W, self.b]
        for h in self.hidden_layers:
            self.params += h.params
            
        # for momentum
        dparams = [theano.shared(np.zeros(p.get_value().shape)) for p in self.params]
        
        # for rmsprop
        cache = [theano.shared(np.zeros(p.get_value().shape)) for p in self.params]
        
        # theano vars
        thX = T.matric('X')
        thY = T.ivecotr('Y')
        pY = self.forward(thX)
        
        # Regularization Cost
        rcost = reg*T.sum([(p*P).sum() for p in self.params])
            

In [8]:
def main():
    X,Y = getData()
    model = ANN([2000,1000])
    model.fit(X,Y, show_fig=True)
    print model.score(X,Y)